# AIPI 590 - xAI

## Interpretable Machine Learning

### Shaila Güereca

# Telco Customer Churn Analysis  

This repository contains the code and analysis for understanding and predicting customer churn using a dataset from a telecommunications company. The dataset can be found [here](https://www.kaggle.com/datasets/blastchar/telco-customer-churn/code). The goal is to identify factors contributing to churn and build interpretable models to predict customers at risk of leaving the company.  

## Instructions  

This project involves the following tasks:  

### 1. **Exploratory Data Analysis (EDA) to Check Assumptions**  
- Perform exploratory analysis of the dataset to examine relationships between features and the target variable (churn).  
- Use appropriate visualizations and statistical methods to assess whether assumptions for linear regression, logistic regression, and Generalized Additive Models (GAM) are met.  

### 2. **Linear Regression**  
- Treat the churn variable as a continuous variable (e.g., 0 for staying, 1 for churning).  
- Build a linear regression model to predict churn.  
- Interpret the coefficients and evaluate the model's performance.  

### 3. **Logistic Regression**  
- Treat churn as a binary variable.  
- Build a logistic regression model to predict the probability of churn.  
- Interpret the coefficients.  

### 4. **Generalized Additive Model (GAM)**  
- Build a GAM to model non-linear relationships between customer features and churn.  
- Interpret the results of the GAM model.  

### 5. **Model Comparison**  
- Compare the performance and interpretability of the linear regression, logistic regression, and GAM models.  
- Discuss the strengths and weaknesses of each approach.  
- Provide recommendations on which model(s) the telecommunications company should use to address their customer churn problem.  



In [2]:
# Loading the required libraries
import os
import pandas as pd
import numpy as np

In [4]:
# Importing the dataset for the same folder as this notebook
data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Displaying the first 5 rows of the dataset
data.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
